# Introduction: Partition Labeling

In this notebook, we will work with a single partition to develop a labeling function. The end goal is code that can take a partition on disk and generate labels with the cutoff time dataframe. This will then be parallelized using Dask.

In [1]:
import pandas as pd 
import numpy as np

import featuretools as ft

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [148]:
PARTITION = '100'
base_dir = '/data/churn/partitions/'
directory = base_dir + 'p' + PARTITION

import os
os.listdir(directory)

['logs.csv',
 'members.csv',
 'train.csv',
 'cancel_cutoff_times.csv',
 'test.csv',
 'transactions.csv',
 'cutoff_times.csv']

In [149]:
all_partitions = os.listdir('/data/churn/partitions/')
len(all_partitions)

1000

### Read in Data Files

The only data we need for making the labels is the transactions. However, we'll read in all the data here.

In [4]:
members = pd.read_csv(f'{directory}/members.csv', 
                      parse_dates=['registration_init_time'], infer_datetime_format = True)
trans = pd.read_csv(f'{directory}/transactions.csv',
                   parse_dates=['transaction_date', 'membership_expire_date'], infer_datetime_format = True)
logs = pd.read_csv(f'{directory}/logs.csv', parse_dates = ['date'])

In [5]:
members.head()
trans.head()
logs.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,46r0VsNbxboshxpjyJ91oWdGX8SHzxFuS2mWkEBujD4=,13,29,female,9,2007-04-29
1,0C2o0WrDEiGkjbQtOR8x3U05OVCVLYFKHVRjgPCA0mM=,10,20,male,9,2007-11-07
2,d1UX5bu9bMtb8mzId7VbHFIUa46UO+IElVh9CxbonNQ=,1,0,NaN,9,2015-01-28
3,mF6w9kCGjtrI0PtOomXQTmN027pGL21K8E2Jvitb0RE=,1,0,NaN,9,2016-12-31
4,EjxD7eoFZ1+/jsVZc+8J79lRbHpZK+ZtyRGZFtZ0at4=,1,0,NaN,4,2017-01-17


,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,G7TmHc9Gg2t8ovG/KFaB53We/0CQPELhZ5UUN2Ol3AQ=,39,30,149,149,1,2015-09-30,2015-11-13,0
1,LPbp8N7VRuqEISEVim8ppTaeYJG/rWS/t4g/dEFuWjw=,34,30,149,149,1,2016-02-29,2016-03-31,0
2,xvYqULBWzJvN8heyFtY3hbY3egyQNbXuDx0igtsoi00=,29,30,180,180,1,2017-01-31,2017-03-01,0
3,UR4iin4mAkajoa7o+AyTTmz5k3N2GR3/rZY8a4KwADI=,41,30,99,99,1,2017-01-31,2017-02-28,0
4,ax8CRhY8BMRA/ZvT1wI+2N/EdPXiSPGxa9y7bntA1Uc=,40,30,149,149,1,2016-05-04,2016-06-08,0


,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,zraYK27dg2odAmzM1z3r4DKqx/9X6P/O7f3VQbTF3zU=,2017-03-18,9,0,1,0,22,26,5714.520
1,Cv+9XMTbZ6Ua6TZoJSJJl7gtkstlXa3R/LHl2zdK+PY=,2017-03-09,4,0,0,0,28,30,3701.484
2,6bL/KHOSEXgB9yeqNpwAZDc9LYd7I/JbUUpXpTAyyKI=,2017-03-21,1,2,1,2,15,16,4288.633
3,UL9bm0eoKgK6YQWCGwz8CtD/9ySFy1fbFBzqUF2jmEs=,2017-03-30,13,3,0,0,30,37,8551.879
4,6cPyxd/lWz24stfnNlsWbW839l5GVWCQ/oJzr7ZT4a4=,2017-03-24,45,9,6,3,32,81,11342.642


## Remove Anomalies

There are a number of records in the transactinos where the `membership_expire_date` is before the `transaction_date`. We will remove these to prevent them from creating issues with our labeling.

In [6]:
trans = trans.loc[trans['membership_expire_date'] >= trans['transaction_date']]

Some of the transactions occur on the same day for the same customer. In these cases, we will assume that the transaction with the later `membership_expire_date` happened second. 

In [104]:
trans = trans.sort_values(['msno', 'transaction_date', 'membership_expire_date'])

# View value counts of customers
trans.groupby('msno').count().sort_values('is_cancel').tail()

,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
msno,,,,,,,,
5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,39,39,39,39,39,39,39,39
CNVTd5cNoA6WRxMf7VveVwYyrOoDgFs5xmm1122qQNc=,39,39,39,39,39,39,39,39
kA7JMc6Q6nAFw1fgcs0hejCw4xXaGUzB/eq+M5n21wQ=,40,40,40,40,40,40,40,40
mFsXs71TCuJOnHKWQQ271BdecYkrXgPIRU7VUfmtqAY=,42,42,42,42,42,42,42,42
/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,51,51,51,51,51,51,51,51


We'll work with the data from one customer at first. 

In [37]:
cust = trans.loc[trans['msno'] == '/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM='].copy()
cust.iloc[:, 1:].tail(6)

,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
12519,41,30,129,129,1,2016-11-08,2016-12-08,0
7776,41,30,129,129,1,2016-11-14,2016-11-14,1
1504,41,30,99,99,1,2016-11-14,2016-12-13,0
5462,41,30,99,99,1,2016-12-13,2017-01-13,0
10717,41,30,99,99,1,2017-01-13,2017-02-13,0
21899,41,30,99,99,1,2017-03-13,2017-04-13,0


In [38]:
cancel = cust.loc[cust['is_cancel'] == 1].copy()
cancel

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
7776,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,129,129,1,2016-11-14,2016-11-14,1


## Find Churns

To find a churn, we'll set a variable `periods` that defines the number of days required without a membership before a customer is considered churned. Then, if there are any gaps in membership longer than this period, we will mark it as a churn. To find the time period without a membership, we'll look at each `membership_expire_date` and calculate the time to the next transaction that is not a cancel. If this value is greater than `periods`, then the customer has churned.

In [39]:
cust.reset_index(inplace = True, drop = True)
periods = 30

In [42]:
for tup in cust.itertuples():
    break
tup

Pandas(Index=0, msno='/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=', payment_method_id=41, payment_plan_days=30, plan_list_price=149, actual_amount_paid=149, is_auto_renew=1, transaction_date=Timestamp('2015-01-13 00:00:00'), membership_expire_date=Timestamp('2016-12-15 00:00:00'), is_cancel=0)

In [52]:
gaps = []

# Iterate through each entry
for tup in cust.itertuples():
    i = tup[0]
    expire_date = tup[8]
    if i == len(cust) - 1:
        gaps.append(np.nan)
    else:
        next_trans = cust.loc[i+1, :].copy()
        j = 0
        while next_trans['is_cancel'] == 1:
            next_trans = cust.loc[i + j, :].copy()
            j += 1
            # Handle the case where the last transaction is cancelled
            if i + j >= len(cust):
                gaps.append(np.nan)
            
        next_start_date = next_trans['transaction_date']
        gaps.append((next_start_date - expire_date).days)

In [86]:
cust['gap'] = gaps

This customer does not have any churns if we define churns as a period of 30 days without renewing. However, they would have one churn if measuring by for example, two weeks. 

The next step is to find out when the churn actually occurs. We'll use two weeks as the churn period so we actually have one churn.

In [58]:
# Days without membership to be considered a churn
periods = 14

# Determine if churned occur
cust['churn'] = cust['gap'] > periods
cust['potential_churn_date'] = cust['membership_expire_date'] + pd.Timedelta(periods, unit = 'd')

# If customer did churn set the churn date
cust.loc[cust['churn'] == 1, 'churn_date'] = cust.loc[cust['churn'] == 1, 'potential_churn_date']
cust.head()
cust.tail()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,gap,potential_churn_date,churn,churn_date
0,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,149,149,1,2015-01-13,2016-12-15,0,-694.0,2016-12-29,False,NaT
1,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,149,119,1,2015-01-21,2017-01-15,0,-720.0,2017-01-29,False,NaT
2,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,149,149,1,2015-01-26,2017-02-15,0,-733.0,2017-03-01,False,NaT
3,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,149,149,1,2015-02-13,2017-03-15,0,-753.0,2017-03-29,False,NaT
4,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,149,119,1,2015-02-21,2017-04-12,0,-776.0,2017-04-26,False,NaT


,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,gap,potential_churn_date,churn,churn_date
46,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,129,129,1,2016-11-14,2016-11-14,1,0.0,2016-11-28,False,NaT
47,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,99,99,1,2016-11-14,2016-12-13,0,0.0,2016-12-27,False,NaT
48,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,99,99,1,2016-12-13,2017-01-13,0,0.0,2017-01-27,False,NaT
49,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,99,99,1,2017-01-13,2017-02-13,0,28.0,2017-02-27,True,2017-02-27
50,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,41,30,99,99,1,2017-03-13,2017-04-13,0,NaN,2017-04-27,False,NaT


# Make Labels

The actual labels are going to occur either on a monthly or two-week basis. For the labels, at the `cutoff_time`, we'll ask two questions: does the customer churn in the next [two weeks / month] and how long until the customer churns.

## Month Labels

We'll start by making labels for each month that the customer was a member.

In [63]:
first_trans = cust['transaction_date'].min()
last_trans = cust['membership_expire_date'].max()
start_date = pd.datetime(first_trans.year, first_trans.month, 1)
end_date = pd.datetime(last_trans.year, last_trans.month, 1)

# Create a range of months
date_range = pd.date_range(start_date, end_date, freq = 'MS')
date_range[:5]
date_range[-5:]

DatetimeIndex(['2015-01-01', '2015-02-01', '2015-03-01', '2015-04-01',
               '2015-05-01'],
              dtype='datetime64[ns]', freq='MS')

DatetimeIndex(['2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01',
               '2019-10-01'],
              dtype='datetime64[ns]', freq='MS')

In [83]:
labels = pd.DataFrame({'cutoff_time': date_range})
labels['next_cutoff_time'] = labels['cutoff_time'].shift(-1)

previous_churn = None

# Iterate through the churn dates
for churn_date in cust.loc[cust['churn_date'].notnull(), 'churn_date']:
    print(churn_date)
    # Assign the label 1 if the customer churned during the cutoff_time period
    labels.loc[(labels['cutoff_time'] <= churn_date) & (labels['next_cutoff_time'] > churn_date), 'churn'] = 1
    
    # If there was a previous churn
    if previous_churn is not None:
        # Subset to cutoff times after the previous churn but before the current churn
        # Calculate the days until the churn
        labels.loc[(labels['cutoff_time'] > previous_churn) & 
                   (labels['cutoff_time'] <= churn_date), 
                   'days_to_next_churn'] = (churn_date - labels.loc[(labels['cutoff_time'] > previous_churn) & 
                                                                   (labels['cutoff_time'] <= churn_date), 'cutoff_time']).dt.days
    # No previous churn
    else:
        # Subset to cutoff times before the current churn and calculate days until the churn
        labels.loc[labels['cutoff_time'] <= churn_date, 
                   'days_to_next_churn'] = (churn_date - labels.loc[labels['cutoff_time'] <= churn_date,
                                                                     'cutoff_time']).dt.days
    previous_churn = churn_date

2017-02-27 00:00:00


In [84]:
labels[labels['churn'] == 1]

,cutoff_time,next_cutoff_time,previous_cutoff_time,churn,days_to_next_churn
25,2017-02-01,2017-03-01,2017-01-01,1.0,26.0


Those are the month labels. Let's write a function that calculates the month labels for one customer.

In [165]:
def generate_month_labels(customer_id, trans, churn_period = 30, return_cust = False):
    """Make labels for one customer for one month
    Params
    --------
        customer_id (str): string used to select customer
        churn_period (int): number of days without membership required for a churn
        
    Return
    --------
        labels (dataframe): labels for all months in customer history
    """
    cust = trans.loc[trans['msno'] == customer_id].copy()
    
    # Make sure to sort transactions
    cust = cust.sort_values(['transaction_date', 'membership_expire_date']).reset_index(drop = True)
    gaps = []

    # Iterate through each entry
    for tup in cust.itertuples():
        i = tup[0]
        expire_date = tup[8]
        if i == len(cust) - 1:
            gaps.append(np.nan)
        else:
            next_trans = cust.loc[i+1, :].copy()
            j = 0
            last_cancelled = False
            while next_trans['is_cancel'] == 1:
                # Handle the case where the last transaction is cancelled
                if i + j >= len(cust):
                    last_cancelled = True
                    gaps.append(np.nan)
                else:
                    next_trans = cust.loc[i + j, :].copy()
                    j += 1
                

            next_start_date = next_trans['transaction_date']
            if not last_cancelled:
                gaps.append((next_start_date - expire_date).days)

    cust['gap'] = gaps
    
    # Determine if churned occur
    cust['churn'] = cust['gap'] > churn_period
    
    # Calculate date range for labels
    first_trans = cust['transaction_date'].min()
    last_trans = cust['membership_expire_date'].max()
    start_date = pd.datetime(first_trans.year, first_trans.month, 1)
    if last_trans.month == 12:
        end_date = pd.datetime(last_trans.year + 1, 1, 1)
    else:
        end_date = pd.datetime(last_trans.year, last_trans.month + 1, 1)

    # Create a range of month starts
    date_range = pd.date_range(start_date, end_date, freq = 'MS')
    
    # Create a label dataframe
    labels = pd.DataFrame({'cutoff_time': date_range})
    labels['next_cutoff_time'] = labels['cutoff_time'].shift(-1)
    labels['msno'] = customer_id
    
    # Handle case where there are no churns
    if not np.any(cust['churn'] == 1):
        labels['churn'] = 0
        labels['days_to_next_churn'] = np.nan
        return labels[['msno', 'cutoff_time', 'churn', 'days_to_next_churn']]
    
    # If customer did churn set the churn date
    cust['potential_churn_date'] = cust['membership_expire_date'] + pd.Timedelta(churn_period, unit = 'd')
    cust.loc[cust['churn'] == 1, 'churn_date'] = cust.loc[cust['churn'] == 1, 'potential_churn_date']
    
    previous_churn = None

    # Iterate through the churn dates
    for churn_date in cust.loc[cust['churn_date'].notnull(), 'churn_date']:
        # Assign the label 1 if the customer churned during the cutoff_time period
        labels.loc[(labels['cutoff_time'] <= churn_date) & (labels['next_cutoff_time'] > churn_date), 'churn'] = 1

        # If there was a previous churn
        if previous_churn is not None:
            # Subset to cutoff times after the previous churn but before the current churn
            # Calculate the days until the churn
            labels.loc[(labels['cutoff_time'] > previous_churn) & 
                       (labels['cutoff_time'] <= churn_date), 
                       'days_to_next_churn'] = (churn_date - labels.loc[(labels['cutoff_time'] > previous_churn) & 
                                                                       (labels['cutoff_time'] <= churn_date), 'cutoff_time']).dt.days
        # No previous churn
        else:
            # Subset to cutoff times before the current churn and calculate days until the churn
            labels.loc[labels['cutoff_time'] <= churn_date, 
                       'days_to_next_churn'] = (churn_date - labels.loc[labels['cutoff_time'] <= churn_date,
                                                                         'cutoff_time']).dt.days
        previous_churn = churn_date
        
    labels['churn'] = labels['churn'].fillna(0)
    
    if return_cust:
        return cust, labels
    
    return labels[['msno', 'cutoff_time', 'churn', 'days_to_next_churn']]

In [126]:
cutoff_times = generate_month_labels('/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=', trans, 14)
cutoff_times[cutoff_times['churn'] == 1].head()

,msno,cutoff_time,churn,days_to_next_churn
25,/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=,2017-02-01,1.0,26.0


In [127]:
cutoff_times = generate_month_labels('/7/KMLZlMBnmWtb9NNkm3bYMQHWrt0C1BChb62EiQLM=',  trans, 30)
cutoff_times[cutoff_times['churn'] == 1].head()

,msno,cutoff_time,churn,days_to_next_churn


In [128]:
cutoff_times = generate_month_labels('5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=', trans)
cutoff_times.head()

,msno,cutoff_time,churn,days_to_next_churn
0,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-01-01,0,NaN
1,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-02-01,0,NaN
2,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-03-01,0,NaN
3,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-04-01,0,NaN
4,5fPXqLcScoC93rH/gCPK+5Soj+XdNMXX9S3LhV5dJjM=,2015-05-01,0,NaN


In [ ]:
cutoff_list = []

# Iterate through every customer
for customer_id in trans['msno'].unique():
    cutoff_list.append(generate_month_labels(customer_id), trans, 30)
    
cutoff_times = pd.concat(cutoff_list)
cutoff_times.head()
cutoff_times.tail()

In [139]:
cust, c = generate_month_labels('0NRMdOljNJEsUC6WtOVzjqdSAdIIdZ1G0Ye6pIlms5U=', trans, return_cust = True)

In [135]:
cust, c = generate_month_labels('+6UN6VJD8u9vZm4lZRAREpzBRM4YoeOSWhEX0c5JBAU=', trans, return_cust = True)

# All Labels for Partition 

The next step is to write a function that can make the month labels for a partition. It will accept a partition number and generate the partition labels. These will then be saved to the partition directory.

In [150]:
partitions = list(range(len(os.listdir('/data/churn/partitions/'))))
partitions[-1]

999

In [166]:
def partition_to_labels(partition, label_type = 'month'):
    trans_file = f'{base_dir}{partition}/transactions.csv'
    print(trans_file)
    trans = pd.read_csv(f'{base_dir}p{partition}/transactions.csv',
                       parse_dates=['transaction_date', 'membership_expire_date'], 
                        infer_datetime_format = True)

    
    cutoff_list = []

    if label_type == 'month':
        # Iterate through every customer
        for customer_id in trans['msno'].unique():
            cutoff_list.append(generate_month_labels(customer_id, trans, 30))
        cutoff_times = pd.concat(cutoff_list)
        cutoff_times.to_csv(f'{base_dir}p{partition}/month_labels.csv', index = False)
        
        

In [ ]:
partition_to_labels(1)

/data/churn/partitions/p1/transactions.csv


In [163]:
partition_to_labels(50)

/data/churn/partitions/p50/transactions.csv


## Potential Outcomes from a Transaction

At the time of the transaction, we want to make a prediction of the behavior following the transaction. We'll establish four different possibilities (five including no data):

* 0: Sign another deal before membership expires
* 1: Sign another deal on the day membership expires
* 2: Sign another deal within 30 days of membership expiring
* 3: Do not sign another deal within 30 days of membership expiring
* NaN: no information in 30 days following expiration

In [ ]:
non_cancels = cust.loc[cust['is_cancel'] == 0].copy()
cancels = cust.loc[cust['is_cancel'] == 1].copy()

## Non Cancellations

First we'll work through making labels for the non-cancellation transactions. A customer is defined as churned if they do not renew within 30 days of the end of the membership. 

In [ ]:
labels = []
label_times = []

# Iterate through each cancellation
for i, transaction in non_cancels.iterrows():
    
    # Find the transaction date and membership renewal date
    transaction_date = transaction['transaction_date']
    expire_date = transaction['membership_expire_date']
        
    label_times.append(transaction_date)
    
    # Customer has 30 days to renew 
    renew_by_date = expire_date + pd.Timedelta(30, 'D')
    
    # Subset to transactions within the renewal period
    renewal_trans = cust.loc[(cust['transaction_date'] >= transaction_date) & 
                             (cust['transaction_date'] <= renew_by_date) & 
                             (cust['is_cancel'] == 0) & (cust.index != i)].copy()
    
    # Data after the end of renewal period
    post_renewal = cust.loc[cust['transaction_date'] > expire_date]
    
    if len(renewal_trans) > 0:
        churned = 0
        
    # No data
    elif (len(renewal_trans) == 0) and (len(post_renewal) == 0):
        churned = np.nan
    
    # Data exists but after renewal period
    elif len(post_renewal) > 0:
        churned = 1
        
    labels.append(churned)

cutoff_times = pd.DataFrame({'cutoff': label_times, 'label': labels})
np.where(np.array(labels) == 1)

In [ ]:
non_cancels.iloc[6:8, 1:]

## Cancellations

We'll do the same thing with the cancellations: for a given cancellation, will the customer sign another contract within 30 days?

In [ ]:
labels = []
label_times = []

# Iterate through each cancellation
for i, transaction in cancels.iterrows():
    
    # Find the transaction date and membership renewal date
    transaction_date = transaction['transaction_date']
    expire_date = transaction['membership_expire_date']
    
    label_times.append(transaction_date)
    
    # Customer has 30 days to renew 
    renew_by_date = expire_date + pd.Timedelta(30, 'D')
    
    # Subset to transactions within the renewal period
    renewal_trans = cust.loc[(cust['transaction_date'] >= transaction_date) & 
                             (cust['transaction_date'] <= renew_by_date) & 
                             (cust['is_cancel'] == 0) & (cust.index != i)].copy()
    
    # Data after the end of renewal period
    post_renewal = cust.loc[cust['transaction_date'] > expire_date]
    
    if len(renewal_trans) > 0:
        churned = 0
        
    # No data
    elif (len(renewal_trans) == 0) and (len(post_renewal) == 0):
        churned = np.nan
    
    # Data exists but after renewal period
    elif len(post_renewal) > 0:
        churned = 1
        
    labels.append(churned)
    
cancel_cutoff_times = pd.DataFrame({'cutoff': label_times, 'label': labels})
np.where(np.array(labels) == 1)

In [ ]:
cancels.iloc[0, 1:]

In [ ]:
cutoff_times.head()
cancel_cutoff_times

# Function to Generate Labels for One Customer

Now we need to take those individual blocks of code and combine them into one function. The input is a customer id and the output is two dataframes: a label cutoff time dataframe for the non-cancellation transactions, and a label cutoff time dataframe for the cancellation transactions.

In [ ]:
trans = pd.read_csv(f'{directory}/transactions.csv',
                   parse_dates=['transaction_date', 'membership_expire_date'], infer_datetime_format = True)

trans = trans.sort_values(['msno', 'transaction_date', 'membership_expire_date'])
trans = trans.loc[trans['membership_expire_date'] >= trans['transaction_date']]

In [ ]:
def make_labels(df, trans_df, customer_id):
    
    labels = []
    label_times = []
    
    # Iterate through each cancellation
    for i, transaction in df.iterrows():

        # Find the transaction date and membership renewal date
        transaction_date = transaction['transaction_date']
        expire_date = transaction['membership_expire_date']

        label_times.append(transaction_date)

        # Customer has 30 days to renew 
        renew_by_date = expire_date + pd.Timedelta(30, 'D')

        # Subset to transactions within the renewal period
        renewal_trans = trans_df.loc[(trans_df['transaction_date'] >= transaction_date) & 
                                     (trans_df['transaction_date'] <= renew_by_date) & 
                                     (trans_df['is_cancel'] == 0) & (trans_df.index != i)].copy()

        # Data after the end of renewal period
        post_renewal = trans_df.loc[trans_df['transaction_date'] > expire_date]

        if len(renewal_trans) > 0:
            churned = 0

        # No data
        elif (len(renewal_trans) == 0) and (len(post_renewal) == 0):
            churned = np.nan

        # Data exists but after renewal period
        elif len(post_renewal) > 0:
            churned = 1

        labels.append(churned)

    cutoff_times = pd.DataFrame({'msno': customer_id,'cutoff': label_times, 'label': labels})
    return cutoff_times

In [ ]:
def make_customer_labels(customer_id, transactions):
    """Make label cutoff time dataframes for a customer. 
    
    Params
    --------
        customer_id (str): customer id (`msno`) as a string
    
    Returns
    --------
        cutoff_times (dataframe): label dataframe with columns 'msno', 'cutoff', and 'label' for the non-cancellation
                                  transactions
        cancel_cutoff_times (dataframe): label dataframe with columns 'msno', 'cutoff', and 'label' for the
                                         cancellation transactions
    """
    # Subset to the customer data
    cust = transactions.loc[transactions['msno'] == customer_id].copy()
    
    non_cancels = cust.loc[cust['is_cancel'] == 0].copy()
    cancels = cust.loc[cust['is_cancel'] == 1].copy()
    
    cutoff_times = make_labels(non_cancels, cust, customer_id)
    cancel_cutoff_times = make_labels(cancels, cust, customer_id)
    
    return cutoff_times, cancel_cutoff_times

In [ ]:
cutoff_times.head()

In [ ]:
ct, cancel_ct = make_customer_labels('ineWmhVlalmqq5V4rNEd8ECLRSdVSwYX7yuxmVSzIsY=', trans)
ct.head()
cancel_ct

## Function to Make Labels for Entire Partition

The next step is to take the function from a single individual to an entire partition. Then, we can run this over every partition.

In [ ]:
def make_partition_labels(partition, save_data = True):
    """Make label cutoff time dataframe for one partition.
    Parameters
    --------
        partition (int): number of partition
        
    Returns
    --------
        No return. Saves results to the partition directory as cutoff_times.csv and cancel_cutoff_times.csv"""
    
    # Read in transactions
    transactions = pd.read_csv(f'/data/churn/partitions/p{partition}/transactions.csv', 
                              parse_dates=['transaction_date', 'membership_expire_date'], 
                               infer_datetime_format = True)
    # Sort and filter
    transactions = transactions.sort_values(['msno', 'transaction_date', 'membership_expire_date'])
    transactions = transactions[transactions['membership_expire_date'] >= transactions['transaction_date']]
    
    # Unique customers
    customer_ids = list(transactions['msno'].unique())
    
    # Lists to hold dataframes
    cutoff_times = []
    cancel_cutoff_times = []
    
    for customer_id in customer_ids:
        ct, cancel_ct = make_customer_labels(customer_id, transactions)
        
        cutoff_times.append(ct)
        cancel_cutoff_times.append(cancel_ct)
    
    # Optionally return the data
    if not save_data:
        return cutoff_times, cancel_cutoff_times
    
    # By default save the data as a single dataframe
    cutoff_times = pd.concat(cutoff_times)
    cutoff_times.to_csv(f'/data/churn/partitions/p{partition}/cutoff_times.csv', index = False)

    cancel_cutoff_times = pd.concat(cancel_cutoff_times)
    cancel_cutoff_times.to_csv(f'/data/churn/partitions/p{partition}/cancel_cutoff_times.csv', index = False)

In [ ]:
ct, cancel_ct = make_partition_labels(500, save_data = False)

In [ ]:
cutoff_times_partition = pd.concat(ct)
cutoff_times_partition.head()

cancel_cutoff_times_partition = pd.concat(cancel_ct)
cancel_cutoff_times_partition.head()

In [ ]:
cutoff_times_partition.shape
cancel_cutoff_times_partition.shape

In [ ]:
len(trans) == (len(cutoff_times_partition) + len(cancel_cutoff_times_partition))

# Use Dask to Parallelize Making Cutoff Times

In [ ]:
partitions = list(range(len(os.listdir('/data/churn/partitions'))))
len(partitions)

In [ ]:
import dask.bag as db
from dask.distributed import Client

# Use all cores
client = Client(processes = True)

In [ ]:
client.ncores()

In [ ]:
# Create a bag object
b = db.from_sequence(partitions, npartitions=len(partitions))

# Map partition making function
b = b.map(make_partition_labels)
    
b

In [ ]:
%%capture
from timeit import default_timer as timer

start = timer()
b.compute()
end = timer()



In [ ]:
print(f'{round(end - start)} seconds elapsed.')

# Conclusions

In this notebook, we generated labels and cutoff time dataframes for each partition. We can then use these cutoff times in a call to deep feature synthesis to make features for each label. 

The next notebook is `Feature Engineering`. 